In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [5]:
search = TavilySearchResults()
search.invoke("what is the weather in Ahmedabad")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Ahmedabad', 'region': 'Gujarat', 'country': 'India', 'lat': 23.0333, 'lon': 72.6167, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1730968889, 'localtime': '2024-11-07 14:11'}, 'current': {'last_updated_epoch': 1730968200, 'last_updated': '2024-11-07 14:00', 'temp_c': 33.1, 'temp_f': 91.6, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 303, 'wind_dir': 'WNW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 32, 'cloud': 0, 'feelslike_c': 31.5, 'feelslike_f': 88.7, 'windchill_c': 34.8, 'windchill_f': 94.6, 'heatindex_c': 33.6, 'heatindex_f': 92.4, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 4.0, 'vis_miles': 2.0, 'uv': 4.7, 'gust_mph': 6.7, 'gust_kph': 10.8}}"},
 {'url': 'https://www.hindustantimes.com/cities/ahmedabad-weather-today-aqi-and-rai

# Retriever

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [9]:
retriever.invoke("how to upload a dataset")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='PythonTypeScriptfrom langsmith import Client, evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.create_examples(  inputs=[      {"postfix": "to LangSmith"},      {"postfix": "to Evaluations in LangSmith"},  ],  outputs=[      {"output": "Welcome to LangSmith"},      {"output": "Welcome to Evaluations in LangSmith"},  ],  dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):  return {"score": run.outpu

In [10]:
from langchain.tools.retriever import create_retriever_tool

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [12]:
tools = [search, retriever_tool]

In [16]:
from langchain_groq import ChatGroq

# Initialize Groq client (direct initialization)
llm = ChatGroq(model="llama3-70b-8192", temperature=0)

In [15]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is the weather in Ahmedabad"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Ahmedabad'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Ahmedabad', 'region': 'Gujarat', 'country': 'India', 'lat': 23.0333, 'lon': 72.6167, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1730969799, 'localtime': '2024-11-07 14:26'}, 'current': {'last_updated_epoch': 1730969100, 'last_updated': '2024-11-07 14:15', 'temp_c': 34.0, 'temp_f': 93.2, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 303, 'wind_dir': 'WNW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 30, 'cloud': 0, 'feelslike_c': 32.6, 'feelslike_f': 90.7, 'windchill_c': 34.8, 'windchill_f': 94.6, 'heatindex_c': 33.6, 'heatindex_f': 92.4, 'dewpoint_c': 10.9, 'dewpoint_f': 51.6, 'vis_km': 4.0, 'vis_miles': 2.0, 'uv': 4.7, '

{'input': 'what is the weather in Ahmedabad',
 'output': 'The current weather in Ahmedabad is 34°C (93.2°F) with an overcast condition and a feelslike temperature of 32.6°C (90.7°F).'}

In [23]:
agent_executor.invoke({"input": "change port in langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'change port in langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick startOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!
1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in gen

{'input': 'change port in langsmith',
 'output': 'It seems like the tool call yielded a list of search results related to changing the port in LangSmith. Based on the results, it appears that LangSmith uses port 8123 by default, but you can change this by modifying the ports in the langgraph-api volume. Additionally, there are environment variables such as LANGCHAIN_HUB_API_URL and LANGCHAIN_ENDPOINT that can be used to configure the APIs in a self-hosted environment.'}